In [1]:
import pandas as pd
import ipywidgets as widgets
import warnings
warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [4]:
raw=pd.read_csv('C:/Users/WangChengLong/Desktop/python/文本挖掘/data/文本.txt',encoding='gbk',sep='aaa',names=['txt'])
def head(char):
    return char[:1]
def mid(char):
    return char.find('回')
raw['head']=raw.txt.apply(head)
raw['mid']=raw.txt.apply(mid)
raw['len']=raw.txt.apply(len)
chapnum=0
for i in range(len(raw.txt)):
    if raw['head'][i]=='第' and raw['mid'][i]>0 and raw['len'][i]<30:
        chapnum+=1
    if raw['txt'][i]=="附录一:成吉思汗家族":
        chapnum=0
    raw.loc[i,'chap']=chapnum
del raw['len']
del raw['head']
del raw['mid']
rg=raw.groupby('chap')
rg=rg.apply(sum)
text_jy=list(rg.txt)

def generate_text_select(data):
    text_num=len(data)
    label=range(text_num)
    value=[data[i] for i in label]
    text_select=widgets.SelectMultiple(options=list(zip(label,value))+[('ALL',data)],value=data,
                                       continuous_update=False)
    return text_select

text_select=generate_text_select(text_jy)#最终传递的是tuple类型

import jieba
import jieba.analyse as ana
import nltk
import numpy as np
import wordcloud
import matplotlib.pyplot as plt
import numpy as np
from imageio import imread

out_cloud=widgets.Output(layout=widgets.Layout(width='auto',height='auto'))
#out_corpus=widgets.Output(layout=widgets.Layout(width='auto',height='auto'))
@out_cloud.capture(clear_output=True,wait=True)
def fenci(text,library,stopwords,addwords,delwords,fenci_model,fenci_result,font,mask,suggest_freq_cut,suggest_freq_add):
    text=[i for i in text]#转化为列表类型
    if library:
        jieba.load_userdict(library)
    if addwords:#注意不要continuous updating
        jieba.add_word(addwords)#text的形式自定义文本
    if delwords:
        jieba.del_word(delwords)#以top20的文本为选项,multiple
    if stopwords:
        ana.set_stop_words(stopwords)
    stoplist=list(pd.read_csv(stopwords,names=['w'],sep='aaa',encoding='utf-8',engine='python').w)
    if suggest_freq_add:
        for i in suggest_freq_add:
            jieba.suggest_freq(i,True)
    if suggest_freq_cut:
        for i in suggest_freq_cut:
            jieba.suggest_freq(eval(i),True)
    if fenci_result=='分词列表':
        corpus_raw=[eval('jieba.'+fenci_model) for i in text]
        corpus=[[j for j in i if j not in stoplist] for i in corpus_raw]
    if fenci_result=='空格连接的字符串':
        #corpus=[' '.join([j for j in eval('jieba.'+fenci_model) if j not in stoplist]) for i in text]
        corpus_raw=[eval('jieba.'+fenci_model) for i in text]
        corpus=[' '.join(j for j in i if j not in stoplist) for i in corpus_raw]
    data=[j for i in text for j in eval('jieba.'+fenci_model) if j not in stoplist]
    fdist=nltk.FreqDist(data)
    ## 设置图云的属性
    cloud=wordcloud.WordCloud(font_path=font,max_words=5000,min_font_size=0.1,relative_scaling=0.1,width=600,height=600,mode='RGBA',background_color='white',stopwords=stopwords,mask=imread(mask))
    #'C://Windows/Fonts/simhei.ttf'
    cloudimage=cloud.fit_words(fdist)
    imarray=np.array(imread(mask))
    ##获取颜色
    bimColors=wordcloud.ImageColorGenerator(imarray)
    ##重置词云颜色
    cloudimage.recolor(color_func=bimColors)
    plt.figure(figsize=(8,8))
    plt.axis("off")
    plt.imshow(cloudimage)
    plt.show()
    return corpus #返回分好词的文本，列表形式，里面元素可能是分词列表，也可能是空格分隔的分好词的文本。

#widgets定义
import os

suggest_freq_cut=widgets.SelectMultiple(options=[],description='拆分某个长词')
suggest_freq_add=widgets.SelectMultiple(options=[],description='保留某个长词')

label=os.listdir('data/词库')
path=['data/词库/'+i for i in label]
library=widgets.Dropdown(options=list(zip(label,path))+[('','')],value='',description='导入分词词库')

label_stop=os.listdir('data/stopword')
path_stop=['data/stopword/'+i for i in label_stop]
stopwords=widgets.Dropdown(options=list(zip(label_stop,path_stop)),description='导入停用词')

addwords=widgets.Text(description='增加词库词语',continuous_update=False)

#delwords=widgets.SelectMultiple(options=ana.extract_tags(jieba.lcut(pd.read_csv('data/文本.txt',encoding='gbk',sep='aaa',names=['txt'])))+[''],
#                               value='')
delwords=widgets.Text(description='删减词库词语',continuous_update=False)

fenci_result=widgets.Dropdown(options=['分词列表','空格连接的字符串'],value='分词列表')

label_model=['精确模式','全模式','搜索模式']
value_model=['lcut(i)','lcut(i,cut_all=True)','lcut_for_search(i)']
fenci_model=widgets.Dropdown(options=list(zip(label_model,value_model)),value='lcut(i)')

label_font=os.listdir('data/Fonts')
value_font=['data/Fonts/'+i for i in label_font]
font=widgets.Dropdown(options=list(zip(label_font,value_font)),value='data/simhei.ttf')

label_mask=[i for i in os.listdir('data/背景图') if i.endswith('.jpg')]
value_mask=['data/背景图/'+i for i in label_mask]
mask=widgets.Dropdown(options=list(zip(label_mask,value_mask)),value='data/背景图/太极.jpg')

w=widgets.interactive_output(fenci,{'text':text_select,'library':library,'stopwords':stopwords,'addwords':addwords,'delwords':delwords,
                                    'fenci_model':fenci_model,'fenci_result':fenci_result,'font':font,'mask':mask,
                                    'suggest_freq_add':suggest_freq_add,'suggest_freq_cut':suggest_freq_cut})
cloud=widgets.HBox(
[widgets.VBox(children=[font,mask,text_select,library,stopwords,addwords,delwords,fenci_result,fenci_model],layout=widgets.Layout(width='40%')),
widgets.VBox([widgets.HBox([suggest_freq_add,suggest_freq_cut]),out_cloud])
],
layout=widgets.Layout(height='600px')
)

#cloud

In [5]:
cloud